##Using Machine Learning

In [ ]:
#predicting fibonacci with linear regression
import pandas as pd

fibonacci = pd.DataFrame([[1, 1, 2], 
                          [2, 1, 3], 
                          [3, 2, 5],
                          [5, 3, 8],
                          [8, 5, 13]
])

fibonacci.columns = [0, 1, 2]
fibonacci.iloc[0]
fibonacci

import numpy as np
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(fibonacci[[0, 1]], fibonacci[2])
reg.score(fibonacci[[0, 1]], fibonacci[2])

def pred(x):
  return reg.predict(np.array([x]))

list1 = [0, 1]

for k in range(25):
  list1.append(int(np.rint(pred([list1[k], list1[k+1]]))))
print(list1)

In [ ]:
import plotly.express as px
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(14, 8))

x = list(CRL[0])
y = list(CRL[1])
z = list(CRL[2])

fig = px.scatter_3d(CRL, x=x, y=y, z=z, text=CRL.index)
fig.show()

##Using Deep Learning

In [ ]:
import numpy as np
from random import seed
from random import randint
from numpy import array

def create_scaler(*args):
  min_d = np.min(args[0])
  max_d = np.max(args[0])
  for partition in args:
    if np.min(partition) < min_d:
      min_d = np.min(partition)
    if np.max(partition) > max_d:
      max_d = np.max(partition)
  return [min_d, max_d]

def normalizer(scaler, df):
  min_d = scaler[0]
  max_d = scaler[1]
  normalized_df = (df - min_d) / (max_d - min_d)
  return normalized_df

def denormalizer(scaler, normalized_df):
  min_d = scaler[0]
  max_d = scaler[1]
  denormalized_df = normalized_df * (max_d - min_d) + min_d
  return denormalized_df

In [ ]:
fibonacci_cache = {}
def fibonacci_memo(input_value):
  if input_value in fibonacci_cache:
      return fibonacci_cache[input_value]
  if input_value == 1:
          value = 1
  elif input_value == 2:
          value = 1
  elif input_value > 2:           
          value =  fibonacci_memo(input_value -1) + fibonacci_memo(input_value -2)
  fibonacci_cache[input_value] = value
  return value

In [ ]:
#create fibonacci sequence
fibonacci = [0]
for i in range(1, 94):
  fibonacci.append(fibonacci_memo(i))

fibonacci = np.array(fibonacci)
fibonacci[0]

fibonacci = np.asarray(fibonacci).astype('int64')

In [ ]:
#create lag in sequence
import numpy as np

X = list()
y = list()
for iter in range(90):
  X.append(fibonacci[iter:iter+2])
  y.append(fibonacci[iter+2])

X = np.array(X)
y = np.array(y).reshape(90, 1)
print(X, y)

In [ ]:
print(X[5], y[5])

In [ ]:
#standardizing the shape for the neural net
#before scaling, so I do not need futher editing
X = X.reshape(90, 2, 1)
y = y.reshape(90, 1, 1)

#scaling down and saving the scaler
scaler = create_scaler(X, y)
X_ = normalizer(scaler, X)
y_ = normalizer(scaler, y)

#splitting the dataset
from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X_, y_, test_size=0.33, random_state=42)

In [ ]:
X_[15]

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Input
from math import sqrt
from sklearn.metrics import mean_squared_error

#   create LSTM
#***for 1-dimensional output I will be using Dense(1)
#***for 2-dimensional output I will be using Dense(2)
model = Sequential()
#   model.add(LSTM(10, input_shape=(2, 1)))
#   DEPRECATED: I want to add input and dense separately
model.add(Input(shape=(2, 1)))
model.add(LSTM(60))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

#train LSTM
model.fit(X_, y_, epochs=20, batch_size=1, verbose=2)

#prediction
result = model.predict(X_, batch_size=1, verbose=0)

In [ ]:
#denormalize
y = denormalizer(scaler, y_)
result = denormalizer(scaler, result)

#       unnesting 1 layer to compare results
#   to compare them they need to be 2 dimensional
#ex. 1-dimensional output: (330, 1), (330, 1)
#ex. 2-dimensional output: (330, 2), (330, 2)
y = y.reshape(90, 1)
result = result.reshape(90, 1)
print(y.shape, result.shape)

In [ ]:
#calculate error
expected = [x for x in y]
predicted = [x for x in result]
rmse = sqrt(mean_squared_error(expected, predicted))
#print('RMSE: %f' % rmse)

#show some examples
for i in range(20):
  error = (expected[i] - predicted[i])

  #valid for all n-dimensional outputs
  print(expected[i], predicted[i])

  #only valid for one_dimensional output
  #print('Expected=%d, Predicted=%d (err=%d)' % (expected[i], predicted[i], error))